## Bootstrapping for Credentials

In [1]:
import dotenv
dotenv.load_dotenv()

True

### Simple parameterized LLM call

Model + Prompt

In [2]:
# Note: langchain recently refactored their package structure
# As of langchain 0.1 LLM provider specific abstractions live in
# dedicated packages that follow the <langchain_provider> format

# from langchain.chat_models import ChatOpenAI # legacy
from langchain_openai import ChatOpenAI # new way

from langchain.prompts import ChatPromptTemplate

In [3]:
model = ChatOpenAI(
  model="gpt-4-1106-preview"
  # model="gpt-4"
  # model="gpt-3.5-turbo"
)

In [4]:
model.invoke("Tell me an intelligent yet genuinely funny joke about python")

AIMessage(content="Why do Pythons live on land?\n\nBecause they're above C-level.")

In [5]:
prompt = ChatPromptTemplate.from_template("Tell me an intelligent yet genuinely funny joke about {topic}")

In [6]:
prompt.invoke({"topic": "lions"})

ChatPromptValue(messages=[HumanMessage(content='Tell me an intelligent yet genuinely funny joke about lions')])

Important insight:

All the essential building blocks provided by LangChain (models, prompts, retrievers) share a common interface :-)

They all support, amongst others:
```
# invoke
# stream
# batch
```

Knowing that both the _model_, and the _prompt_ support being "invoked", we can do this:

In [7]:
model.invoke(prompt.invoke({"topic": "cats"}))

AIMessage(content="Why don't cats play poker in the jungle?\n\nToo many cheetahs.")

But there's a much more elegant way...

### LangChain Expression Language (LCEL)

LCEL allows us to pipe together multiple of the elemental building blocks into a single object, a so-called _chain_. The resulting chain can then also be invoked, streamed, or batched, analogously to the individual building blocks it is composed of.

In [8]:
chain = prompt | model

In [9]:
chain.invoke({"topic": "snail"})

AIMessage(content='A snail walks into a car dealership and buys a sleek new sports car. He asks the salesman to have a special modification made: he wants a big "S" painted on each side, the roof, and the trunk of the car.\n\nPuzzled, the salesman asks why the snail wants this customization.\n\nThe snail responds, "When I zoom past people on the highway, I want them to say, \'Look at that S-car go!\'"')

Let's add another building block to the chain: an output parser

In [10]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [11]:
chain = prompt | model | output_parser

In [12]:
chain.invoke({"topic": "python"})

'Why do Python programmers prefer dark mode?\n\nBecause light attracts bugs.'

### How can we invoke the chain with a string as input, rather than a full dictionary?

In [13]:
# This will fail:
try:
  chain.invoke("lemons")
except TypeError as err:
  print(err)

Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.


We can use the special `RunnablePassthrough` to maintain the input value but map it to the expected key-value format:

In [14]:
from langchain_core.runnables import RunnablePassthrough

chain_str = {"topic": RunnablePassthrough()} | chain

In [15]:
chain_str.invoke("lemons")

'Why did the lemon stop halfway across the road?\n\nBecause it ran out of juice!'

## Parametrization with more than one parameter

In [16]:
prompt = ChatPromptTemplate.from_template("""
Tell me a joke about {topic} 

Answer in {language}""")

In [17]:
chain_with_multiple_parameters = prompt | model | output_parser

In [18]:
chain_with_multiple_parameters.invoke({"topic": "python", "language": "German"})

'Warum mögen Python-Programmierer keine Umlaute?\n\nWeil sie ständig mit Encodings kämpfen müssen!'

This is a good example to illustrate the power of the `.batch` method that all LCEL objects support. This way multiple calls can be made in parallel:

In [19]:
chain_with_multiple_parameters.batch([{"topic": "lemons", "language": "English"},
                                      {"topic": "python", "language": "German"},
                                      {"topic": "cats", "language": "Dutch"}
                                      ])

['Why did the lemon stop rolling down the hill?\n\nBecause it ran out of juice!',
 'Warum mögen Pythons keine Umlaute?\n\nWeil sie beim Versuch, "Schönheit" zu sagen, immer eine Exception werfen!',
 'Waarom zijn katten slecht in videogames?\n\nOmdat ze altijd met de muis spelen!']

## Stream the output

In [20]:
prompt = ChatPromptTemplate.from_template("""
Write a 100 word love-letter about Python.
""")

chain_long_answer =  prompt | model | output_parser

In [21]:
for token in chain_long_answer.stream({}):
  print(token, end="")

My Dearest Python,

From the first moment I beheld your elegant syntax, my heart was enraptured. You’ve charmed me with your simplicity and ensnared me with your versatility. Each line of your code is a love note, transforming my thoughts into action with effortless grace. Your libraries, like bouquets of functionality, bloom with possibility. You've taught me the true meaning of efficiency, allowing our dance of logic to flow like prose.

In your embrace, I’ve found a sanctuary of creativity where I can express myself with clarity and purpose. My beloved Python, you are the serenade to my algorithmic soul.

Yours in code,
A smitten developer

## Local RAG – Sample Implementation

In [22]:
# from langchain.embeddings import OpenAIEmbeddings # legacy
from langchain_openai import OpenAIEmbeddings # new way

from langchain.vectorstores import Chroma

Let's define some facts and store them in a vectorstore:

In [23]:
vectorstore = Chroma.from_texts([
  "Arno likes walks in nature",
  "Bob loves Python",
  "Mira enjoys working with Prefect",
  "Craig is a big fan of cycling",
],
embedding=OpenAIEmbeddings()
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

We can retrieve the closest fact to a given query:

In [25]:
retriever.invoke("What does Bob like?")

[Document(page_content='Bob loves Python')]

The embeddings encapsulate "meaning", thus we can also retrieve facts that are semantically close to the query:

In [26]:
retriever.invoke("Who has a hobby that involves two wheels?")

[Document(page_content='Craig is a big fan of cycling')]

The embeddings can also infer semantic similarity across languages:

In [27]:
# The question is in German: "Who likes to take walks?"
retriever.invoke("Wer mag Spaziergänge?")

[Document(page_content='Arno likes walks in nature')]

Now we can combine the retriever with a prompt and model to create an LLM that is grounded in our custom knowledge base:

In [28]:
# Optional: Enable debug mode to see the full chain of operations
# from langchain.globals import set_debug
# set_debug(False)

In [29]:
from operator import itemgetter

In [30]:
prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:
{context}

Question:
{question}
""")

Our end goals is to be able to invoke the chain with a question like this:
`chain.invoke({"question": "What do you know about Mira?"})`

We can compose a chain that allows us to do this as follows:

In [31]:
chain_rag = (
        {
          "context": itemgetter("question") | retriever,
          "question": itemgetter("question"),
        }

  | prompt
  | model
  | output_parser
)

Explanation:
The outermost `(` and `)` are simply there to allow us to freely add line-breaks for easier readability.

The final composition of `prompt`, `model`, and `output_parser` are the same as before.

The only difference in the chain definition is the addition of the dictionary at the beginning of the chain, which is used to determine the value of the two keys: `context` and `question` which are expected as input to the prompt we defined above. The values of the dictionary can be LCEL objects themselves, as is the case for the `retriever` in this example.

The usage of `itemgetter` allows us to extract the value of a key from a dictionary. In this case, the chain will be called with a single attribute as input, namely `question`. The `itemgetter` will extract the value of the key `question` from the input dictionary twice. Once to pass it to the retriever to obtain the semantically most relevant facts and store them as the `context`. And a second time to preserve the value of the question and make it available as input to the next step in the chain, the prompt.

In [32]:
for token in chain_rag.stream({"question": "What do you know about Mira?"}):
  print(token, end="")

Based on the provided context, I know that Mira enjoys working with something called Prefect. However, without additional information, it's unclear whether Prefect refers to a person, a job title, a company, a software tool, or something else entirely. The context does not provide any further details about Mira, such as her profession, background, or the nature of her work with Prefect.

This works as intended :-)

We can apply the same trick as before to allow the chain to be invoked with a string as input:

In [33]:
from langchain_core.runnables import RunnablePassthrough

In [34]:
chain_rag_v2 = (
    {"question": RunnablePassthrough()}
  | chain_rag
)

In [35]:
for token in chain_rag_v2.stream("What do you know about Craig? Can you reply with a 50 word essay and make up some more relevant filler information?"):
  print(token, end="")

Craig harbors a deep passion for cycling, a sport that captivates his interest and likely consumes much of his free time. He may spend weekends pedaling through scenic trails or attending cycling events. His dedication suggests he could be an advocate for cycling benefits, possibly encouraging others to embrace the activity.

If we wish to maintain the context from the retriever throughout the whole chain, we can do that in the following way:

In [36]:
chain_rag_with_context = (
  RunnablePassthrough() 
  |
        {
          "context": itemgetter("question") | retriever,
          "question": itemgetter("question"),
        }

  |
        {
          "answer": {"question": itemgetter("question"),
                     "context": itemgetter("context")} 
                    | prompt | model | output_parser,
          "context": itemgetter("context"),
        }
)

In [37]:
chain_rag_with_context.invoke({"question": "What do you know about Arno?"})

{'answer': 'Based on the given context, I know that Arno enjoys taking walks in natural environments.',
 'context': [Document(page_content='Arno likes walks in nature')]}

### Appendix: Why do we use `itemgetter()`?

The `itemgetter()` function is a very simple function that returns a function that can be used to extract the value of a key from a dictionary. It is equivalent to the following lambda function:

```python
lambda x: x["key"]
```

Or as a normal function:

```python
def get_key(x):
  return x["key"]
```


In the context of LCEL if we have a chain of the form <step_a> | <step_b> | <step_c>, then the output of <step_a> is passed as input to <step_b>, and the output of <step_b> is passed as input to <step_c>.

In the example of `"context": itemgetter("question") | retriever,` itemgetter allows us to take the dictionary passed in from the previous step in the chain as input and extract a pre-determined key from it. 

It allows us to keep the same convention, i.e. the previous steps' output can directly be passed as input to the next step.

Example of itemgetter:

In [38]:
from operator import itemgetter

In [39]:
my_list = [1,2,3]

In [40]:
my_list[0]

1

In [41]:
getter = itemgetter(0)

In [42]:
getter(my_list)

1